In [1]:
# reading spare ice netcdf (gzipped) files and outputting monthly files for totals 
#  and counts onto a chosen grid, which can then be read by another
#  python script
# dduncan, 20/12/17
# updated 12/2/18 to output histogram stats to file (changed commented out sections
#  if looking at this later)

import netCDF4
from mpl_toolkits.basemap import Basemap, cm
#from pyhdf.SD import SD, SDC
import gzip
import shutil
import tempfile
import os

In [2]:
def open_netcdf(fname):
    if fname.endswith(".gz"):
        infile = gzip.open(fname, 'rb')
        tmp = tempfile.NamedTemporaryFile(delete=False)
        shutil.copyfileobj(infile, tmp)
        infile.close()
        tmp.close()
        data = netCDF4.Dataset(tmp.name)
        os.unlink(tmp.name)
    else:
        data = netCDF4.Dataset(fname)
    return data

In [3]:
yr = '13'
dirt = '/home/dudavid/Dendrite/Dendrite/SatData/SpareICE/avhrr_345_mhs_345_angles_tsurfcfsr_all_global_h/20'+yr+'/' 
months = sorted(os.listdir(dirt))
print(months)

startlat = 90.0 # abs value of NS boundaries
gsize = 0.5 # grid size in degree
nx, ny = int(360/gsize), int(startlat*2/gsize)


['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']


In [11]:
for m in months[:]:
    #if os.path.isfile(outfi):
    #    print(outfi,' already exists!')
    #else:
    print('month: ',m)
    iwpg,iwpgc = np.zeros([ny,nx]), np.zeros([ny,nx]) # re-init for each month
    iwpall = []
    days = sorted(os.listdir(dirt+m+'/'))
    ndays = 0 #init
    for d in days[:]:
        print(d)
        files = sorted(os.listdir(dirt+m+'/'+d))
        if len(files) != 0:  # in case no files in day dir
            ndays += 1
            for fix in files: #range(len(files)):
                file = dirt+m+'/'+d+'/'+fix
                #print(file)
                nc = open_netcdf(file)
                lon, lat = nc.variables['LON'][:], nc.variables['LAT'][:] 
                iwp = np.array(nc.variables['IWP']) * .001 # into kg
                #mask = [dn == 0] # DAYTIME ONLY
                #iwc = iw[mask,:] # just selected profiles
                #lat = lat[mask]
                #lon = lon[mask]
        
                # calculate lat/lon indices for Xdeg grid (fixed 23/11):
                ladex = np.round((lat[:]+startlat-gsize*.5) / gsize)
                lodex = np.round((lon[:]+180.0-gsize*.5) / gsize)

                scrn = [iwp >= 0.0] #AVOIDs missing (negative) values
                #print(len(iwp),len(iwp[scrn]))
                if len(scrn) == 0: print('problem!')
                siwp = iwp[scrn] # >=0 values only!
                if len(siwp) > 0: 
                    scndscrn = [abs(lat[scrn]) <= 60]
                    iwpall.extend(siwp[scndscrn]) # just take 60NS points for histo
                    #print(np.max(siwp[scndscrn]))
                
                ### commented out for histo/pdf calculation!
                #for x in range(nx):
                #    if np.any(lodex == x):
                #        dexsub = np.where(lodex == x)
                #        subset = iwp[dexsub] 
                #        subsetla = ladex[dexsub]
                #        for y in range(ny):
                #            if np.any(subsetla == y):
                #                lolasubset = np.where(subsetla == y)
                #                iwpg[y,x] += subset[lolasubset].sum() # add up, get mean afterward
                #                iwpgc[y,x] += len(subset[lolasubset])
                                
                            
    # create histogram of IWP
    binz = [2**x for x in range(-14,6)]
    binzz = [-.001,2**(-14)]
    hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
    histaz, bin_edges2= np.histogram(np.array(iwpall),bins=binzz)
    nd = str(ndays)
    # dataset name, res (x,y), time res of input, output arrays (grids, zonals), version?, # days
    #fin = 'grids/spare.'+str(nx)+'x.'+str(ny)+'y.iwptot.cts.hist.v1.'+yr+m+'.'+nd
    fin = 'grids/spare.histo.60NS.v2.'+yr+m+'.'+nd # v2 includes zero counts as first histo element
    #np.save(fin+'.tot',iwpg)
    #np.save(fin+'.cts',iwpgc)
    outz = np.append(histaz,hista) #[histaz,hista]
    print(outz)
    np.save(fin,outz)


month:  01
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
[23006261        1       25       97     1323    11502    41411   179880
   600086  1019714  1169244  1152562  1022946   772770   503820   322814
   192788    75664    18191     3339]
month:  02
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
[20690542        0        0       18      460     4991    26090   146232
   500854   866167   999729  1009839   924144   722048   481641   310634
   180371    69742    17957     3149]
month:  03
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
[22823984        1        2       13      482     3828    24922   168437
   569829   960831  1098297  1135030  1091368   853724   548309   349045
   200508    77470    21981     4123]
month:  04
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
[22345599        0        0       12      26